In [1]:
!pip install transformers==2.6.0

import pandas as pd
import numpy as np

from numpy.random import seed
import tensorflow as tf
import random as rn
import os
from nltk import tokenize 
import nltk
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
nltk.download('punkt')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
import traitlets
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras import backend as K

import gc

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
AUTO = tf.data.experimental.AUTOTUNE

# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
#Transformer implementation is based on the following code: https://www.kaggle.com/miklgr500/jigsaw-tpu-bert-with-huggingface-and-keras

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.94.179.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.94.179.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
CODES_PATH = 'drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-classification'
CV_PATH    = 'drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-classification/models/cv'
TEST_PATH  = 'drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-classification/models/final'

In [4]:
max_features = 7000
maxlen       = 2000
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [5]:
def other(x):
    if ((x=='exchanges')|(x=='finance')|(x=='games')|(x=='gambling')|(x=='high-risk')): 
        return x 
    else: 
        return 'other'

In [6]:
data = pd.read_csv("drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-Dapp-scraping/data/dapps_source_code_parsed_open_source_2021-01-08.csv")

data = data[data.SC_full!='\n']
data.drop('Unnamed: 0', axis=1, inplace=True)
data.category = data.category.apply(other)
data.comments.fillna('empty', inplace=True)

In [7]:
RS = 42
SPLITS = 3
LENGTH = 5000

In [8]:
X = data.drop('category', axis=1)
y = data.category

In [9]:
X, X_test, y, y_test  = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RS)

In [10]:
print(f'X_train shape {X.shape}')
print(f'X_test shape {X_test.shape}')

X_train shape (1142, 12)
X_test shape (286, 12)


## Only The Comments

In [11]:
names  = [f'aucroc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names2 = [f'aucprc_{i}' for i in ['exchanges', 'finance', 'gambling', 'games', 'high-risk']]
names.extend(names2)

kf  = StratifiedKFold(n_splits=SPLITS, random_state=RS, shuffle=True)
TRAIN = True

In [12]:
n_words  = range(500, 1200, 200)
length   = range(500, 1200, 200)
emb_dim  = [50, 100]
n_hidden = [50, 100]
epochs   = [4]
lr       = [0.01, 0.001, 0.0001]
drop     = [0, 0.25]


n_words  = [1100]
length   = [500]
emb_dim  = [50]        
n_hidden = [100]      
drop     = [0]      
epochs   = [4]
lr       = [0.01]

In [13]:
# First load the real tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')

# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer("vocab.txt", lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, add_special_tokens=True, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True, wordpieces_prefix=##)

In [14]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

def build_model(transformer, loss='categorical_crossentropy', max_len=512, lr=lr):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #x = tf.keras.layers.Dropout(0.35)(cls_token)
    out = Dense(5, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(lr=lr),
                  metrics=[tf.keras.metrics.AUC()])
    
    return model

In [15]:
TRAIN = True

#### BERT

In [16]:
for WORD in n_words:
  for LENGTH in length:
    for DIM in emb_dim:
      for HID in n_hidden:
        for DROP in drop:
          for EPOCH in epochs:
            for RATE in lr:

                metrics_cv = pd.DataFrame(np.zeros((SPLITS, len(names))), columns=names)
                for c, (train_index, val_index) in enumerate(kf.split(X, y)):

                        X_train, X_val      = X.iloc[train_index]['comments'], X.iloc[val_index]['comments']
                        y_train, y_val      = y.iloc[train_index], y.iloc[val_index] 

                        le = LabelBinarizer()
                        le.fit(y_train)

                        y_train = le.transform(y_train)[:,0:5]
                        y_val = le.transform(y_val)[:,0:5]
                        
      
                        X_train = fast_encode(X_train, fast_tokenizer, maxlen=LENGTH)
                        X_val = fast_encode(X_val, fast_tokenizer, maxlen=LENGTH)

                        train_dataset = (
                        tf.data.Dataset
                        .from_tensor_slices((X_train, y_train))
                        .shuffle(42)
                        .batch(64)
                        .prefetch(AUTO)
                        )

                        valid_dataset = (
                        tf.data.Dataset
                        .from_tensor_slices((X_val, y_val))
                        .batch(64)
                        .cache()
                        .prefetch(AUTO)
                        )
                        
                        tf.tpu.experimental.initialize_tpu_system(tpu)
                        strategy = tf.distribute.experimental.TPUStrategy(tpu)

                        with strategy.scope():
                          transformer_layer = transformers.TFBertModel.from_pretrained('bert-base-uncased')
                          classifier = build_model(transformer=transformer_layer,  max_len=LENGTH, lr=3e-5)

                        if TRAIN:
                              history = classifier.fit(train_dataset, epochs=4)
                              classifier.save_weights(f'{CV_PATH}/BERT_comments_fold_{c}.h5')
                        else:
                              classifier.load_weights(f'{CV_PATH}/BERT_comments_fold_{c}.h5')

                        probs = classifier.predict(valid_dataset, batch_size=64, verbose=1)

                        assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()
                        
                        for i, cl in enumerate(le.classes_[le.classes_!='other']):
                            auc_roc_class = roc_auc_score(y_val[:,i], probs[:,i])
                            auc_pr_class  = average_precision_score(y_val[:,i], probs[:,i])

                            metrics_cv.loc[c, f'aucroc_{cl}'] = round(auc_roc_class, 3)
                            metrics_cv.loc[c, f'aucprc_{cl}'] = round(auc_pr_class, 3)   
                        
                        del classifier
                        K.clear_session()
                        gc.collect()

INFO:tensorflow:Initializing the TPU system: grpc://10.94.179.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.94.179.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/4


12/12 [==============================] - 97s 252ms/step - loss: 1.2196 - auc: 0.6790
Epoch 2/4
12/12 [==============================] - 3s 255ms/step - loss: 0.8586 - auc: 0.7598
Epoch 3/4
12/12 [==============================] - 3s 254ms/step - loss: 0.7904 - auc: 0.6912
Epoch 4/4
6/6 [==============================] - 18s 1s/step


In [17]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.923,0.717,0.471,0.751,0.590,0.913,0.530,0.046,0.444,0.050
1,0.939,0.694,0.397,0.501,0.573,0.926,0.579,0.040,0.179,0.092
2,0.947,0.771,0.535,0.574,0.561,0.941,0.633,0.052,0.214,0.228


In [18]:
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.94.179.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.94.179.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [19]:
le = LabelBinarizer()
le.fit(y)

y_train = le.transform(y)[:,0:5]
y_test  = le.transform(y_test)[:,0:5]

x_train = fast_encode(X['comments'], fast_tokenizer, maxlen=LENGTH)
x_test  = fast_encode(X_test['comments'], fast_tokenizer, maxlen=LENGTH)

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .shuffle(42)
    .batch(64)
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_test, y_test))
    .batch(64)
    .cache()
    .prefetch(AUTO)
)

In [20]:
%%time
with strategy.scope():
    transformer_layer = transformers.TFBertModel.from_pretrained('bert-base-uncased')
    classifier = build_model(transformer_layer, max_len=LENGTH, lr=3e-5)

metrics_cv = pd.DataFrame(np.zeros((1, len(names))), columns=names)

if TRAIN:
      history = classifier.fit(train_dataset, epochs=4)
      classifier.save_weights(f'{TEST_PATH}/BERT_comments.h5')
else:
      classifier.load_weights(f'{TEST_PATH}/BERT_comments.h5')

probs = classifier.predict(test_dataset, batch_size=64, verbose=1)

assert (le.classes_ == ['exchanges', 'finance', 'gambling', 'games', 'high-risk', 'other']).all()

for i, cl in enumerate(le.classes_[le.classes_!='other']):
    auc_roc_class = roc_auc_score(y_test[:,i], probs[:,i])
    auc_pr_class  = average_precision_score(y_test[:,i], probs[:,i])

    metrics_cv.loc[0, f'aucroc_{cl}'] = round(auc_roc_class, 3)
    metrics_cv.loc[0, f'aucprc_{cl}'] = round(auc_pr_class, 3)   

del classifier
K.clear_session()
gc.collect()

Epoch 1/4


18/18 [==============================] - 138s 3s/step - loss: 1.1608 - auc: 0.6782
Epoch 2/4
18/18 [==============================] - 5s 255ms/step - loss: 0.7812 - auc: 0.7174
Epoch 3/4
18/18 [==============================] - 5s 254ms/step - loss: 0.6519 - auc: 0.6748
Epoch 4/4
5/5 [==============================] - 19s 3s/step
CPU times: user 1min 10s, sys: 8.17 s, total: 1min 19s
Wall time: 3min 49s


In [21]:
metrics_cv

,aucroc_exchanges,aucroc_finance,aucroc_gambling,aucroc_games,aucroc_high-risk,aucprc_exchanges,aucprc_finance,aucprc_gambling,aucprc_games,aucprc_high-risk
0,0.968,0.777,0.353,0.789,0.602,0.965,0.637,0.038,0.628,0.056
